# Demo - Constraint Based Synthesis

Dependencies
<!-- - OPTIONAL: [Docker](https://www.docker.com/) -->
- Python 3ish
- [Jupyter](https://jupyter.org/)
- [Racket](https://racket-lang.org/)
- [Rosette](https://docs.racket-lang.org/rosette-guide/index.html): constraint based solver based on racket

Setup (No Docker)
1. Download & install [Racket](https://racket-lang.org/), add to path
2. Run:
    ```sh
    # optional but recommended: virtualenv
    python3 -m venv .venv
    source .venv/bin/activate
    pip install -r requirements.txt  # this installs jupyter

    # install rosette
    raco pkg install rosette

    jupyter notebook
    ```
3. Change environment variables at the top of your target notebook as required, as you may need to add racket to the venv path manually

In [1]:
# Julia -- this is for my machine
import os

# Add /Applications/Racket v8.16/bin to PATH
os.environ['PATH'] = '/Applications/Racket v8.16/bin:$PATH'
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib:$DYLD_LIBRARY_PATH'

#todos

Dan --> linear function with examples, verbose mode

Julia --> simple arithmetic DSL with `define-grammar` & sketching

# Example 1
## _Linear Function Synthesis from Examples_

Find a function of the form $f(x) = Ax + B$ that satisfies the following examples: 
- $f(1) = 5$
- $f(2) = 7$
- $f(3) = 9$
- $f(4) = 11$

---
```racket
#lang rosette
(require rosette/lib/synthax) ; the package containing `synthesis` syntax

; Define a function `f(x) = Ax + B`, with unknown coefficients `A` and `B` as `??`
(define (f x)
  (+ (* (?? integer?) x) (?? integer?)))  ; `??` means a hole in the program

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5)  (2 . 7)  (3 . 9)  (4 . 11)))

; Synthesis constraint: f(x) must match expected output
(define sol
  (synthesize
   #:forall (list)
   #:guarantee
   (begin
     (for/list ([pair examples])
       (assert (= (f (car pair)) (cdr pair))))))) ; for every pair (x, y) in `examples` assert f(x) = y

(print-forms sol)
```


In [2]:
!racket racket/linear-func-synth.rkt

/Users/julianonn/pv703/cs703-rosette/racket/linear-func-synth.rkt:9:0
(define (f x) (+ (* 2 x) 3))


__Example 1 - Result:__   $f(x) = 2x + 3$

In [3]:
# Example 1 -- Verbose Mode
!racket racket/linear-func-synth-VERBOSE.rkt

(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(declare-fun c0 () Int)
(declare-fun c1 () Int)
(define-fun e2 () Int (+ c0 c1))
(define-fun e3 () Bool (= 5 e2))
(define-fun e4 () Int (* 2 c0))
(define-fun e5 () Int (+ c1 e4))
(define-fun e6 () Bool (= 7 e5))
(define-fun e7 () Bool (and e3 e6))
(define-fun e8 () Int (* 3 c0))
(define-fun e9 () Int (+ c1 e8))
(define-fun e10 () Bool (= 9 e9))
(define-fun e11 () Bool (and e7 e10))
(define-fun e12 () Int (* 4 c0))
(define-fun e13 () Int (+ c1 e12))
(define-fun e14 () Bool (= 11 e13))
(define-fun e15 () Bool (and e11 e14))
(assert e15)
(check-sat)
(get-model)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(reset)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :

---
---

# Example 2 

Let's make this slightly more complicated and create a DSL (domain-specific language) for arithmetic over integers that includes addition, multiplication, and squaring.

--- 

```racket
#lang rosette

(require rosette/lib/synthax) ; the package containing `synthesis` syntax
(require rosette/lib/destruct)

; Let's define a simple grammar for arithmetic expressions over integers
; with addition,  multiplication, and squaring
(struct Add  (left right) #:transparent)
(struct Mult (left right) #:transparent)
(struct Square (arg) #:transparent)

(define (interpret expr)
  (destruct expr
    [(Add a b)    (+ (interpret a) (interpret b))]
    [(Mult a b)   (* (interpret a) (interpret b))]
    [(Square a)   (expt (interpret a) 2)]
    [_ expr]))


; Define a function f(x) = Ax + B
(define (f x)
  (Add (Mult (?? integer?) x) (?? integer?)))  ; `??` means a hole in the program

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5)  (2 . 7)  (3 . 9)  (4 . 11)))

; Synthesis constraint: f(x) must match expected output
(define sol
  (synthesize
   #:forall (list)
   #:guarantee
   (begin
     (for/list ([pair examples])
       (assert (= (interpret (f (car pair))) (cdr pair)))))))  ; for every pair (x, y) in `examples` assert f(x) = y

(print-forms sol)
```


In [4]:
!racket racket/arith-dsl-1.rkt

/Users/julianonn/pv703/cs703-rosette/racket/arith-dsl-1.rkt:21:0
(define (f x) (Add (Mult 2 x) 3))


__Example 2 - Result:__   $f(x) = 2x + 3$

In [5]:
# Example 2 -- Verbose Mode
!racket racket/arith-dsl-1-VERBOSE.rkt

(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(declare-fun c0 () Int)
(declare-fun c1 () Int)
(define-fun e2 () Int (+ c0 c1))
(define-fun e3 () Bool (= 5 e2))
(define-fun e4 () Int (* 2 c0))
(define-fun e5 () Int (+ c1 e4))
(define-fun e6 () Bool (= 7 e5))
(define-fun e7 () Bool (and e3 e6))
(define-fun e8 () Int (* 3 c0))
(define-fun e9 () Int (+ c1 e8))
(define-fun e10 () Bool (= 9 e9))
(define-fun e11 () Bool (and e7 e10))
(define-fun e12 () Int (* 4 c0))
(define-fun e13 () Int (+ c1 e12))
(define-fun e14 () Bool (= 11 e13))
(define-fun e15 () Bool (and e11 e14))
(assert e15)
(check-sat)
(get-model)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(reset)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :

---
---

# Example 3

Using the same DSL, synthesize a quadratic expression of the form $Ax^2 + B$ from examples $f(2) = 9, f(3) = 19$.


---

```racket
; ... [same DSL code] ...

; Define a function f(x) = A * x^2 + B
(define (f x) (Add (Mult (?? integer?) (Square x)) (?? integer?))) 

; Examples (x . y), meaning f(x) = y
(define examples
  `((1 . 5) (2 . 9)  (3 . 19)))

; ...
```

In [6]:
!racket racket/arith-dsl-2.rkt

/Users/julianonn/pv703/cs703-rosette/racket/arith-dsl-2.rkt:21:0
(define (f x) (Add (Mult 2 (Square x)) 1))


__Example 3 - Result__: $f(x) = 2x^2 + 1$

In [7]:
# Example 3 -- Verbose mode
!racket racket/arith-dsl-2-VERBOSE.rkt

(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(declare-fun c0 () Int)
(declare-fun c1 () Int)
(define-fun e2 () Int (* 4 c1))
(define-fun e3 () Int (+ c0 e2))
(define-fun e4 () Bool (= 9 e3))
(define-fun e5 () Int (* 9 c1))
(define-fun e6 () Int (+ c0 e5))
(define-fun e7 () Bool (= 19 e6))
(define-fun e8 () Bool (and e4 e7))
(assert e8)
(check-sat)
(get-model)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(reset)
(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
/Users/julianonn/pv703/cs703-rosette/racket/arith-dsl-2-VERBOSE.rkt:24:0
(define (f x) (Add (Mult 2 (Square x)) 1))


---
--- 

# Example 4

Let's experiment with more complicated **sketching**. In previous examples, we synthesized functions of a certain forms (i.e. $f(x) = Ax + ...$), but since we defined this form explicitly for the solver, we only _really_  synthesized integer coefficients.

Before, we used `??` to represent integer "holes" that the solver should synthesize. 

Here we define a custom "hole" `??expr` to represent unknown arithmetic expressions of the following forms:

- $a + b$, $a*b$, $a$,  for some integer terminals $a, b$.
- _Note:_  this doesn't currently include $a + (a + b)$ and $a^2$


#### Task: 
Find two expressions $f, g$ such that $f + g = 10x$




---

```racket
; ... [same DSL code] ...

; In order to do `sketching`, we need to define constraints on an unknown expression `??expr` that we want to synthesize

(define (??expr terminals)
  (define l (apply choose* terminals))       ; given a list of args, `choose*` returns a value that can evaluate to any of them
  (define r (apply choose* terminals))       
  (choose* (Add l r) (Mult l r) l))

;  For instance, `??expr (list 2 x))` could return (Add 2 x), (Mult 2 x),  2, or x


; Use sketching to synthesize an expression of the form `f + g` equal to `10x`
(define-symbolic x c1 c2 integer?)  
(define sketch (Add (??expr (list x c1 c2)) (??expr (list x c1 c2))))

(define M      ; model
  (synthesize
    #:forall (list x)
    #:guarantee (assert (= (interpret sketch)
                           (interpret (Mult 10 x))))))
(evaluate sketch M)

```

In [8]:
!racket racket/arith-dsl-sketch.rkt

(Add (Mult 15 x) (Mult x -5))


__Example 4 - Result:__ 

$f := 15x,~ g := -5x ~\longrightarrow~ 15x + -5x = 10x$

&nbsp;

Bonus: the result changes if we change the order of the `choose*` statements in the source code.

In [9]:
# Example 4 -- Verbose mode
!racket racket/arith-dsl-sketch-VERBOSE.rkt

(set-option :auto-config true)
(set-option :produce-unsat-cores false)
(set-option :smt.mbqi.max_iterations 10000000)
(set-option :smt.relevancy 2)
(declare-fun c0 () Bool)
(declare-fun c1 () Bool)
(declare-fun c2 () Int)
(declare-fun c3 () Bool)
(declare-fun c4 () Int)
(declare-fun c5 () Int)
(define-fun e6 () Int (ite c3 c4 c5))
(define-fun e7 () Int (ite c1 c2 e6))
(declare-fun c8 () Bool)
(declare-fun c9 () Bool)
(define-fun e10 () Int (ite c9 c4 c5))
(define-fun e11 () Int (ite c8 c2 e10))
(define-fun e12 () Int (+ e7 e11))
(declare-fun c13 () Bool)
(define-fun e14 () Bool (not c0))
(define-fun e15 () Bool (and c13 e14))
(define-fun e16 () Int (* e7 e11))
(define-fun e17 () Bool (not c13))
(define-fun e18 () Bool (and e14 e17))
(define-fun e19 () Int (+ (ite c0 e12 0) (ite e15 e16 0) (ite e18 e7 0)))
(declare-fun c20 () Bool)
(declare-fun c21 () Bool)
(declare-fun c22 () Bool)
(define-fun e23 () Int (ite c22 c4 c5))
(define-fun e24 () Int (ite c21 c2 e23))
(declare-fun c25 () Bool